In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:

import xarray as xr
import pandas as pd
import numpy as np
import string


# plotting
from cartopy import crs as ccrs , feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import ListedColormap
from matplotlib import colors
import cmocean as cm
from cmcrameri import cm as cmf


# user defined functions
import utils_OM as om



In [3]:
#%% load data
t0=2003;t1=2016
t0_a=1993;t1_a=2016
path='../data/output/'

ds=xr.open_dataset(path+'final_dataset_{}-{}.nc'.format(t0,t1))
ds_a=xr.open_dataset(path+'final_dataset_{}-{}.nc'.format(t0_a,t1_a))

lon=np.array(ds.lon)
lat=np.array(ds.lat)
X=np.array(lon);X[-1]=360
Y=np.array(-lat)



#% % mask
mask=xr.open_dataset(path+'../LAND_MASK.CRI_360x180.nc')
# mask.land_mask.plot()
mask=mask.sortby('lat',ascending=False)
oceanmask=np.array(mask.land_mask)
oceanmask[oceanmask>0]=np.nan
oceanmask[oceanmask==0]=1


In [4]:
df_glb = pd.read_pickle(path+'global-final_dataset-{}-{}_v2.p'.format(t0,t1))
df_glb_combo = pd.read_pickle(path+'global-final_combos-{}-{}_v2.p'.format(t0,t1))

df_glb_a = pd.read_pickle(path+'global-final_dataset-{}-{}_v2.p'.format(t0_a,t1_a))
df_glb_combo_a = pd.read_pickle(path+'global-final_combos-{}-{}_v2.p'.format(t0_a,t1_a))


In [5]:
# https://www.tablesgenerator.com/#

# GSML Tables - Annex

In [6]:
df=df_glb[['region','dataset','trend','sum_unc']]
# df['trend'] = df_glb['trend']
df = df.rename(columns={'sum_unc': 'unc'})

df_glb_combo = df_glb_combo.rename(columns={'combinations':'region','total_unc':'unc'})
df_glb_combo['dataset']=df_glb_combo['region']
df_glb_combo = df_glb_combo.drop(df_glb_combo.columns[4:len(df_glb_combo.columns)],axis=1)
# df_glb_combo


In [7]:
df2=df.append(df_glb_combo, ignore_index=True)
df2 = df2.rename(columns={'trend':'2003 trend','unc':'2003 unc'})
df2

,region,dataset,2003 trend,2003 unc
0,AIS,AIS_CSR,0.311081,0.109505
1,AIS,AIS_IMB,0.338398,0.117834
2,AIS,AIS_JPL,0.273927,0.113901
3,AIS,AIS_UCI,0.510031,0.104780
4,GIS,GIS_CSR,0.736490,0.087953
5,GIS,GIS_IMB,0.619243,0.086911
6,GIS,GIS_JPL,0.751503,0.088124
7,GIS,GIS_UCI,0.816629,0.082383
8,GLA,GLA_CSR,0.357522,0.152251
9,GLA,GLA_JPL,0.393303,0.152694


In [8]:
df=df_glb_a[['region','dataset','trend','sum_unc']]
# df['trend'] = df_glb['trend']
df = df.rename(columns={'sum_unc': 'unc'})

df_glb_combo_a = df_glb_combo_a.rename(columns={'combinations':'region','total_unc':'unc','datasets':'dataset'})
df_glb_combo_a = df_glb_combo_a.drop(df_glb_combo_a.columns[4:len(df_glb_combo_a.columns)],axis=1)
df_glb_combo_a['dataset']=df_glb_combo_a['region']


df3=df.append(df_glb_combo_a, ignore_index=True)
df3 = df3.rename(columns={'trend':'1993 trend','unc':'1993 unc'})


In [9]:
df3

,region,dataset,1993 trend,1993 unc
0,AIS,AIS_IMB,0.276238,0.090668
1,AIS,AIS_UCI,0.391492,0.086563
2,GIS,GIS_IMB,0.432852,0.058265
3,GIS,GIS_UCI,0.503882,0.063947
4,GLA,GLA_WGP,0.485937,0.155700
5,GLA,GLA_ZMP,0.268439,0.154216
6,LWS,LWS_GWB,0.323654,0.078049
7,LWS,LWS_WGP,0.232697,0.077028
8,JPL,JPL,0.000000,0.000000
9,CSR,CSR,0.000000,0.000000


In [10]:
df4 = pd.merge(df2,df3,on=['dataset','region'],how='outer')
df4
#df4['ord'] = [0,1]

,region,dataset,2003 trend,2003 unc,1993 trend,1993 unc
0,AIS,AIS_CSR,0.311081,0.109505,NaN,NaN
1,AIS,AIS_IMB,0.338398,0.117834,0.276238,0.090668
2,AIS,AIS_JPL,0.273927,0.113901,NaN,NaN
3,AIS,AIS_UCI,0.510031,0.104780,0.391492,0.086563
4,GIS,GIS_CSR,0.736490,0.087953,NaN,NaN
5,GIS,GIS_IMB,0.619243,0.086911,0.432852,0.058265
6,GIS,GIS_JPL,0.751503,0.088124,NaN,NaN
7,GIS,GIS_UCI,0.816629,0.082383,0.503882,0.063947
8,GLA,GLA_CSR,0.357522,0.152251,NaN,NaN
9,GLA,GLA_JPL,0.393303,0.152694,NaN,NaN


In [11]:
df4['order']=[0,2,1,3,4,6,5,7,8,9,10,11,12,15,12,14,17,16,18,19,20,21]
df4 = df4.sort_values(by='order')
print(df4)


         region      dataset  2003 trend  2003 unc  1993 trend  1993 unc  \
0           AIS      AIS_CSR    0.311081  0.109505         NaN       NaN   
2           AIS      AIS_JPL    0.273927  0.113901         NaN       NaN   
1           AIS      AIS_IMB    0.338398  0.117834    0.276238  0.090668   
3           AIS      AIS_UCI    0.510031  0.104780    0.391492  0.086563   
4           GIS      GIS_CSR    0.736490  0.087953         NaN       NaN   
6           GIS      GIS_JPL    0.751503  0.088124         NaN       NaN   
5           GIS      GIS_IMB    0.619243  0.086911    0.432852  0.058265   
7           GIS      GIS_UCI    0.816629  0.082383    0.503882  0.063947   
8           GLA      GLA_CSR    0.357522  0.152251         NaN       NaN   
9           GLA      GLA_JPL    0.393303  0.152694         NaN       NaN   
10          GLA      GLA_WGP    0.568915  0.154163    0.485937  0.155700   
11          GLA      GLA_ZMP    0.199029  0.153445    0.268439  0.154216   
12          

In [12]:
df4.to_excel(path+'/gmsl_table.xlsx')


# Spatial average and range of regional values

In [13]:
stats={}
print('2003-2016')
print('                       trend    |      unc     ')

combos = [['JPL'],
            ['CSR'],
            ['IMB','WGP'],
           ['IMB','GWB','ZMP'],
          ['UCI','WGP'],['UCI','GWB','ZMP']
         ]
reconstr =['JPL',
             'CSR',
             'IMB+WGP',
            'IMB+GWB+ZMP','UCI+WGP','UCI+GWB+ZMP'
          ]
i=0
ci=0.9
mask=np.array(oceanmask)

for name in np.array(ds.name):
    trend=np.array(ds.trend.sel(name=name))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    unc=np.array(ds.unc_total.sel(name=name))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
    
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(name,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))



    
    print('--')

    
    stats['{}'.format(name)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(mid,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_mid,unc_range_low,unc_range_up)
                               }
    
for title,combo in zip(reconstr,combos):

    names=[name for name in np.array(ds.name) for comb in combo if name.split('_')[1]==comb ]
        #% % 
    trend=np.nansum(np.array(ds.trend.sel(name=names)),axis=0)
    unc=om.quadrsum(np.array(ds.unc_total.sel(name=names)))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
    
    
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(title,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))

    
    print('--')

    
    stats['{}'.format(title)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(mid,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_mid,unc_range_low,unc_range_up)
                               }


                              
stats_2003=stats

2003-2016
                       trend    |      unc     
AIS_CSR: 0.3113 [-0.3697, 0.5256]  |   0.1643 [0.1083,0.3963]
--
AIS_IMB: 0.4169 [-0.1621, 0.5346]  |   0.1066 [0.0580,0.3912]
--
AIS_JPL: 0.2656 [-0.3151, 0.4722]  |   0.1861 [0.1235,0.3969]
--
AIS_UCI: 0.4892 [-0.1465, 0.6380]  |   0.0776 [0.0132,0.3892]
--
GIS_CSR: 0.7161 [-0.6113, 0.9676]  |   0.0818 [0.0444,0.1414]
--
GIS_IMB: 0.1649 [-0.2229, 0.2150]  |   0.0300 [0.0207,0.1320]
--
GIS_JPL: 0.7124 [-0.6185, 0.9590]  |   0.0836 [0.0452,0.1435]
--
GIS_UCI: 0.7829 [-0.6437, 1.0473]  |   0.0292 [0.0184,0.1313]
--
GLA_CSR: 0.3528 [-0.0831, 0.4398]  |   0.0904 [0.0613,0.2772]
--
GLA_JPL: 0.3907 [-0.1161, 0.4850]  |   0.1075 [0.0724,0.2774]
--
GLA_WGP: 0.5685 [-0.4394, 0.6859]  |   0.0965 [0.0645,0.2775]
--
GLA_ZMP: 0.2443 [0.1049, 0.2914]  |   0.1641 [0.0829,0.2807]
--
LWS_CSR: 0.1538 [-0.0234, 0.2656]  |   0.7537 [0.5817,0.8705]
--
LWS_GWB: 0.2823 [0.1515, 0.3906]  |   1.0686 [0.8038,1.2427]
--
LWS_JPL: 0.2779 [0.0073, 0.4090]  

In [14]:
stats_2003=stats
stats_2003

{'AIS_CSR': {'trend': '0.31 [-0.37,0.53]', 'unc': '0.16 [0.11,0.40]'},
 'AIS_IMB': {'trend': '0.42 [-0.16,0.53]', 'unc': '0.11 [0.06,0.39]'},
 'AIS_JPL': {'trend': '0.27 [-0.32,0.47]', 'unc': '0.19 [0.12,0.40]'},
 'AIS_UCI': {'trend': '0.49 [-0.15,0.64]', 'unc': '0.08 [0.01,0.39]'},
 'GIS_CSR': {'trend': '0.72 [-0.61,0.97]', 'unc': '0.08 [0.04,0.14]'},
 'GIS_IMB': {'trend': '0.16 [-0.22,0.21]', 'unc': '0.03 [0.02,0.13]'},
 'GIS_JPL': {'trend': '0.71 [-0.62,0.96]', 'unc': '0.08 [0.05,0.14]'},
 'GIS_UCI': {'trend': '0.78 [-0.64,1.05]', 'unc': '0.03 [0.02,0.13]'},
 'GLA_CSR': {'trend': '0.35 [-0.08,0.44]', 'unc': '0.09 [0.06,0.28]'},
 'GLA_JPL': {'trend': '0.39 [-0.12,0.48]', 'unc': '0.11 [0.07,0.28]'},
 'GLA_WGP': {'trend': '0.57 [-0.44,0.69]', 'unc': '0.10 [0.06,0.28]'},
 'GLA_ZMP': {'trend': '0.24 [0.10,0.29]', 'unc': '0.16 [0.08,0.28]'},
 'LWS_CSR': {'trend': '0.15 [-0.02,0.27]', 'unc': '0.75 [0.58,0.87]'},
 'LWS_GWB': {'trend': '0.28 [0.15,0.39]', 'unc': '1.07 [0.80,1.24]'},
 'LWS_JP

In [15]:
stats={}
stats_1993={}
print('1993-2016')
print('                       trend    |      unc     ')

combos = [
            ['IMB','WGP'],
           ['IMB','GWB','ZMP'],
          ['UCI','WGP'],['UCI','GWB','ZMP']
         ]
reconstr =[
             'IMB+WGP',
            'IMB+GWB+ZMP','UCI+WGP','UCI+GWB+ZMP'
          ]
i=0
ci=0.9
mask=np.array(oceanmask)

for name in np.array(ds_a.name):
    
    #% %
    unc_glb=np.array(df_glb_a.loc[df_glb_a['dataset'].isin([name])]['sum_unc'])[0]
    tr_glb= np.array(df_glb_a.loc[df_glb_a['dataset'].isin([name])]['trend'])[0] 
    
    trend=np.array(ds_a.trend.sel(name=name))
    unc=np.array(ds_a.unc_total.sel(name=name))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
        
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(name,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))

    
    print('--')

    
    stats['{}'.format(name)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(tr_glb,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_glb,unc_range_low,unc_range_up)
                               }
                              
        
for title,combo in zip(reconstr,combos):
 
    names=[name for name in np.array(ds_a.name) for comb in combo if name.split('_')[1]==comb ]
        #% % 
    trend=np.nansum(np.array(ds_a.trend.sel(name=names)),axis=0)
    unc=om.quadrsum(np.array(ds_a.unc_total.sel(name=names)))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
        
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(title,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))


    
    print('--')
    
    stats['{}'.format(title)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(mid,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_mid,unc_range_low,unc_range_up)
                               }
                              
stats_1993=stats

1993-2016
                       trend    |      unc     
AIS_IMB: 0.1703 [-0.0661, 0.2182]  |   0.1672 [0.0974,0.5372]
--
AIS_UCI: 0.4137 [-0.1228, 0.5377]  |   0.1171 [0.0370,0.5302]
--
GIS_IMB: 0.3576 [-0.4352, 0.4678]  |   0.0655 [0.0302,0.0978]
--
GIS_UCI: 0.5207 [-0.4408, 0.6953]  |   0.0660 [0.0305,0.0886]
--
GLA_WGP: 0.4772 [-0.3206, 0.5750]  |   0.0656 [0.0454,0.1301]
--
GLA_ZMP: 0.0004 [-0.0003, 0.0005]  |   0.0171 [0.0021,0.1253]
--
LWS_GWB: 0.3532 [0.2383, 0.4259]  |   0.6403 [0.4881,0.7419]
--
LWS_WGP: 0.2507 [0.1461, 0.3071]  |   0.4317 [0.3323,0.4966]
--
IMB+WGP: 1.2559 [-0.3860, 1.5013]  |   0.4931 [0.3654,0.7491]
--
IMB+GWB+ZMP: 0.8815 [0.0545, 1.0692]  |   0.6859 [0.5069,0.9036]
--
UCI+WGP: 1.6624 [-0.1605, 2.0002]  |   0.4830 [0.3512,0.7445]
--
UCI+GWB+ZMP: 1.2880 [0.2165, 1.5864]  |   0.6789 [0.4970,0.8998]
--


In [16]:
stats_all = {'2003':stats_2003,
            '1993':stats_1993}
stats_all

{'2003': {'AIS_CSR': {'trend': '0.31 [-0.37,0.53]', 'unc': '0.16 [0.11,0.40]'},
  'AIS_IMB': {'trend': '0.42 [-0.16,0.53]', 'unc': '0.11 [0.06,0.39]'},
  'AIS_JPL': {'trend': '0.27 [-0.32,0.47]', 'unc': '0.19 [0.12,0.40]'},
  'AIS_UCI': {'trend': '0.49 [-0.15,0.64]', 'unc': '0.08 [0.01,0.39]'},
  'GIS_CSR': {'trend': '0.72 [-0.61,0.97]', 'unc': '0.08 [0.04,0.14]'},
  'GIS_IMB': {'trend': '0.16 [-0.22,0.21]', 'unc': '0.03 [0.02,0.13]'},
  'GIS_JPL': {'trend': '0.71 [-0.62,0.96]', 'unc': '0.08 [0.05,0.14]'},
  'GIS_UCI': {'trend': '0.78 [-0.64,1.05]', 'unc': '0.03 [0.02,0.13]'},
  'GLA_CSR': {'trend': '0.35 [-0.08,0.44]', 'unc': '0.09 [0.06,0.28]'},
  'GLA_JPL': {'trend': '0.39 [-0.12,0.48]', 'unc': '0.11 [0.07,0.28]'},
  'GLA_WGP': {'trend': '0.57 [-0.44,0.69]', 'unc': '0.10 [0.06,0.28]'},
  'GLA_ZMP': {'trend': '0.24 [0.10,0.29]', 'unc': '0.16 [0.08,0.28]'},
  'LWS_CSR': {'trend': '0.15 [-0.02,0.27]', 'unc': '0.75 [0.58,0.87]'},
  'LWS_GWB': {'trend': '0.28 [0.15,0.39]', 'unc': '1.07 [

In [17]:
reform = {(outerKey, innerKey): values 
          for outerKey, innerDict in stats_all.items()
          for innerKey, values in innerDict.items()}
df = pd.DataFrame.from_dict(reform, orient='index').transpose()
df.columns = pd.MultiIndex.from_tuples(df.columns)
print(df.T)

                              trend               unc
2003 AIS_CSR      0.31 [-0.37,0.53]  0.16 [0.11,0.40]
     AIS_IMB      0.42 [-0.16,0.53]  0.11 [0.06,0.39]
     AIS_JPL      0.27 [-0.32,0.47]  0.19 [0.12,0.40]
     AIS_UCI      0.49 [-0.15,0.64]  0.08 [0.01,0.39]
     GIS_CSR      0.72 [-0.61,0.97]  0.08 [0.04,0.14]
     GIS_IMB      0.16 [-0.22,0.21]  0.03 [0.02,0.13]
     GIS_JPL      0.71 [-0.62,0.96]  0.08 [0.05,0.14]
     GIS_UCI      0.78 [-0.64,1.05]  0.03 [0.02,0.13]
     GLA_CSR      0.35 [-0.08,0.44]  0.09 [0.06,0.28]
     GLA_JPL      0.39 [-0.12,0.48]  0.11 [0.07,0.28]
     GLA_WGP      0.57 [-0.44,0.69]  0.10 [0.06,0.28]
     GLA_ZMP       0.24 [0.10,0.29]  0.16 [0.08,0.28]
     LWS_CSR      0.15 [-0.02,0.27]  0.75 [0.58,0.87]
     LWS_GWB       0.28 [0.15,0.39]  1.07 [0.80,1.24]
     LWS_JPL       0.28 [0.01,0.41]  0.95 [0.73,1.09]
     LWS_WGP       0.26 [0.12,0.32]  0.80 [0.59,0.93]
     JPL          1.65 [-0.43,2.19]  0.98 [0.77,1.16]
     CSR          1.53 [-0.3

In [18]:
df = pd.DataFrame.from_dict(stats_2003).transpose()
df = df.rename(columns={'trend':'2003 trend', 'unc': '2003 unc'})
df       
df['dataset']=df.index
df2 = pd.DataFrame.from_dict(stats_1993).transpose()
df2 = df2.rename(columns={'trend':'1993 trend', 'unc': '1993 unc'})
df2  
df2['dataset']=df2.index


In [19]:

df4 = pd.merge(df,df2,on='dataset',how='outer')
df4
df4['order']=[0,2,1,3,4,6,5,7,8,9,10,11,12,15,12,14,17,16,18,19,20,21]
df4 = df4.sort_values(by='order')
df4 = df4.set_index('dataset')


In [20]:
df4.to_excel(path+'/regional_mean_range_table.xlsx')


In [21]:
stats={}
stats_1993={}
print('1993-2016')
print('                       trend    |      unc     ')

combos = [
            ['IMB','WGP'],

         ]
reconstr =[
            'IMB+GWB+ZMP','UCI+WGP','UCI+GWB+ZMP'
          ]
i=0
ci=0.9
mask=np.array(oceanmask)

for name in np.array(ds_a.name):
    
    #% %
    unc_glb=np.array(df_glb_a.loc[df_glb_a['dataset'].isin([name])]['sum_unc'])[0]
    tr_glb= np.array(df_glb_a.loc[df_glb_a['dataset'].isin([name])]['trend'])[0] 
    
    trend=np.array(ds_a.trend.sel(name=name))
    unc=np.array(ds_a.unc_total.sel(name=name))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
        
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(name,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))
    # glb=oceanmean(trend*oceanmask)


    
    print('--')

    
    stats['{}'.format(name)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(tr_glb,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_glb,unc_range_low,unc_range_up)
                               }
                              
        
for title,combo in zip(reconstr,combos):
 
    names=[name for name in np.array(ds_a.name) for comb in combo if name.split('_')[1]==comb ]
        #% % 
    trend=np.nansum(np.array(ds_a.trend.sel(name=names)),axis=0)

    unc=om.quadrsum(np.array(ds_a.unc_total.sel(name=names)))
    _,mid,_=om.reg_to_glb(trend,np.array(ds.lat),np.array(ds.lon))
    _,unc_mid,_=om.reg_to_glb(unc,np.array(ds.lat),np.array(ds.lon))
        
    x = np.array(trend*mask)
    x = x[np.isfinite(x)]
    range_low,range_up = np.percentile(np.hstack(x),[5,95])
    x = np.array(unc*mask)
    x = x[np.isfinite(x)]
    unc_range_low,unc_range_up = np.percentile(np.hstack(x),[5,95])
    
    print('{}: {:.4f} [{:.4f}, {:.4f}]  |   {:.4f} [{:.4f},{:.4f}]'.format(title,mid,range_low,range_up,
                                               unc_mid,unc_range_low,unc_range_up))

    
    print('--')
    
    stats['{}'.format(title)]={'trend':'{:.2f} [{:.2f},{:.2f}]'.format(mid,range_low,range_up),

                          
                               'unc':'{:.2f} [{:.2f},{:.2f}]'.format(unc_mid,unc_range_low,unc_range_up)
                               }
                              
stats_1993=stats

1993-2016
                       trend    |      unc     
AIS_IMB: 0.1703 [-0.0661, 0.2182]  |   0.1672 [0.0974,0.5372]
--
AIS_UCI: 0.4137 [-0.1228, 0.5377]  |   0.1171 [0.0370,0.5302]
--
GIS_IMB: 0.3576 [-0.4352, 0.4678]  |   0.0655 [0.0302,0.0978]
--
GIS_UCI: 0.5207 [-0.4408, 0.6953]  |   0.0660 [0.0305,0.0886]
--
GLA_WGP: 0.4772 [-0.3206, 0.5750]  |   0.0656 [0.0454,0.1301]
--
GLA_ZMP: 0.0004 [-0.0003, 0.0005]  |   0.0171 [0.0021,0.1253]
--
LWS_GWB: 0.3532 [0.2383, 0.4259]  |   0.6403 [0.4881,0.7419]
--
LWS_WGP: 0.2507 [0.1461, 0.3071]  |   0.4317 [0.3323,0.4966]
--
IMB+GWB+ZMP: 1.2559 [-0.3860, 1.5013]  |   0.4931 [0.3654,0.7491]
--
